### ***Build a Simple LLM Application with LCEL - LangChain Expression Language***

***Objective***

***Build a simple LLM application with LangChain which will translate text from English into another language. It's a single LLM call plus some prompting.***

- *Using language models*
- *Using PromptTemplates and OutputParsers*
- *Using LangChain Expression Language (LCEL) to chain components together*
- *Debugging and tracing your application using LangSmith*
- *Deploying your application with LangServe*

In [ ]:
### Open AI API Key and Open Source models--Llama3,Gemma2,mistral--Groq

import os
from dotenv import load_dotenv
load_dotenv()

import openai
openai.api_key = os.getenv("OPENAI_API_KEY")
groq_api_key = os.getenv("GROQ_API_KEY")

In [4]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
model = ChatGroq(model = "llama-3.1-8b-instant", groq_api_key = groq_api_key)
model

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 8192, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x1522346d0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x1530c1570>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [5]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English to French"),
    HumanMessage(content="Hello How are you?")
]

result = model.invoke(messages)

In [6]:
result

AIMessage(content='Bonjour Comment allez-vous ?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 47, 'total_tokens': 54, 'completion_time': 0.012316943, 'completion_tokens_details': None, 'prompt_time': 0.002678345, 'prompt_tokens_details': None, 'queue_time': 0.010522855, 'total_time': 0.014995288}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_9ca2574dca', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b23ad-f884-76b0-8e12-178aabe51c24-0', usage_metadata={'input_tokens': 47, 'output_tokens': 7, 'total_tokens': 54})

In [7]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()
parser.invoke(result)

'Bonjour Comment allez-vous ?'

In [8]:
### Using LCEL- chain the components
chain = model | parser
chain.invoke(messages)

'Bonjour Comment allez-vous ?'

In [9]:
### Prompt Templates
from langchain_core.prompts import ChatPromptTemplate

generic_template = "Translate the following into {language}:"

prompt = ChatPromptTemplate.from_messages(
    [("system", generic_template), ("user", "{text}")]
)

In [13]:
result = prompt.invoke({"language":"French","text":"Hello"})
result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into French:', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})])

In [11]:
result.to_messages()

[SystemMessage(content='Translate the following into French:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})]

In [12]:
##Chaining together components with LCEL
chain = prompt | model | parser
chain.invoke({"language":"French", "text":"Hello"})

'Bonjour'